In [1]:
import xarray as xr
import netCDF4 as nc

In [2]:
import dask.dataframe as dd

In [3]:
from dask.distributed import Client, LocalCluster

In [4]:
import matplotlib.pyplot as plt

In [5]:
import numpy as np

In [6]:
import os
import time

In [7]:
from testcases import testcases

In [8]:
USERNAME = "x-twhite77"
NUM_REPETITIONS = 5
NUM_WORKERS = 64
MEMORY = "512GB"

In [9]:
client = None

In [10]:
def initialize() ->None:
    cluster = LocalCluster(n_workers=NUM_WORKERS, memory_limit=1/(NUM_WORKERS + 1))
    client = Client(cluster)

def shutdown() ->None:
    client.close()

In [11]:
ds = xr.open_mfdataset("/anvil/datasets/ncar/AORC_Forcing/2016/201601*", engine = "netcdf4")

In [12]:
ds

<xarray.Dataset>
Dimensions:         (time: 744, reference_time: 1, x: 4608, y: 3840)
Coordinates:
  * time            (time) datetime64[ns] 2016-01-01 ... 2016-01-31T23:00:00
  * reference_time  (reference_time) datetime64[ns] 2016-01-01
  * x               (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y               (y) float64 -1.92e+06 -1.919e+06 ... 1.918e+06 1.919e+06
Data variables:
    crs             (time) |S1 b'' b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b''
    U2D             (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    V2D             (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    LWDOWN          (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    RAINRATE        (time, y, x) float32 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    T2D             (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    Q2D             (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    PSFC            (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    SWDOWN          (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    LQFRAC          (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
Attributes:
    model_output_valid_time:    2016-01-01_00:00:00
    model_initialization_time:  2016-01-01_00:00:00
    NWM_version_number:         v2.1
    model_configuration:        AORC
    model_output_type:          forcing
    model_total_valid_times:    744.0

In [13]:
dataset = ds.chunk({"time":24*7, "x":800,"y":1000})

In [14]:
dataset

<xarray.Dataset>
Dimensions:         (time: 744, reference_time: 1, x: 4608, y: 3840)
Coordinates:
  * time            (time) datetime64[ns] 2016-01-01 ... 2016-01-31T23:00:00
  * reference_time  (reference_time) datetime64[ns] 2016-01-01
  * x               (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y               (y) float64 -1.92e+06 -1.919e+06 ... 1.918e+06 1.919e+06
Data variables:
    crs             (time) |S1 dask.array<chunksize=(168,), meta=np.ndarray>
    U2D             (time, y, x) float64 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
    V2D             (time, y, x) float64 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
    LWDOWN          (time, y, x) float64 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
    RAINRATE        (time, y, x) float32 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
    T2D             (time, y, x) float64 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
    Q2D             (time, y, x) float64 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
    PSFC            (time, y, x) float64 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
    SWDOWN          (time, y, x) float64 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
    LQFRAC          (time, y, x) float64 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
Attributes:
    model_output_valid_time:    2016-01-01_00:00:00
    model_initialization_time:  2016-01-01_00:00:00
    NWM_version_number:         v2.1
    model_configuration:        AORC
    model_output_type:          forcing
    model_total_valid_times:    744.0

In [15]:
dataset['RAINRATE']

<xarray.DataArray 'RAINRATE' (time: 744, y: 3840, x: 4608)>
dask.array<rechunk-merge, shape=(744, 3840, 4608), dtype=float32, chunksize=(168, 1000, 800), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2016-01-01 ... 2016-01-31T23:00:00
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
Attributes:
    cell_methods:    time: mean
    remap:           remapped via ESMF regrid_with_weights: Bilinear
    grid_mapping:    crs
    esri_pe_string:  PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DAT...
    proj4:           +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1=30.0...
    units:           mm s^-1
    standard_name:   precipitation_flux
    long_name:       Surface Precipitation Rate

In [ ]:
dataset.to_zarr("/anvil/scratch/x-twhite77/name.zarr")

/home/x-twhite77/.conda/envs/rocky8.7/conda-4.12.0/labpacks/lib/python3.9/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 54.86 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
2024-04-19 10:21:19,210 - distributed.worker.memory - WARNING - Worker is at 89% memory usage. Pausing worker.  Process memory: 3.31 GiB -- Worker memory limit: 3.69 GiB
2024-04-19 10:21:19,353 - distributed.worker.memory - WARNING - Worker is at 54% memory usage. Resuming worker. Process memory: 2.00 GiB -- Worker memory limit: 3.69 GiB
2024-04-19 10:21:19,356 - distributed.worker.memory - WARNING - Worker is at 82% memory usage. Pausing worker.  Process memory: 3.03 GiB -- Worker memory limit: 3.69 GiB
2024-04-19 10:21:19,398 - distributed.worker.memory - WARNING - Worker is at 83% memory usage. Pausing worker.  Process memory: 3.08 GiB -- Worker memory limit: 3.69 GiB
2024-04-19 10:21:19,792 - distributed.worker.me

		Test case: Max_Rain_Test
		Test case: Average_200_sqr
		Test case: Average_400_sqr
		Test case: Average_800_sqr
		Test case: Average_1600_sqr


In [12]:
def GetTestTime(datatset, testcases) -> int:
  times = []

  for func in testcases:
    print(f"\t\tTest case: {func['name']}")
    timetoadd = 0
    startTime = time.time()

    for i in range(NUM_REPETITIONS):
      func['func'](datatset)
      
    endTime = time.time()
    timetoadd += endTime - startTime
    timetoadd = timetoadd / NUM_REPETITIONS

    times.append(timetoadd)
    
  return times


In [13]:
def RunTestCases(pictureName, testcases, chunkSchemes) -> None:
  print("Running benchmark test cases")
  times = []

  for scheme in chunkSchemes:
    fullpath = os.path.join(f"/anvil/scratch/{USERNAME}/", f"{scheme['name']}.zarr")
    print(f"\tTesting scheme: {scheme['name']}")
    ds = xr.open_zarr(fullpath)
    times.append(GetTestTime(ds, testcases))

  times = np.array(times)

In [14]:
 for idx, test in enumerate(testcases):
    plt.figure(figsize=(12, 6))

    # Create a line plot
    plt.plot([scheme['name'] for scheme in chunkSchemes], times[:, idx])

    # Add labels and title
    plt.xlabel('Chunk Size')
    plt.ylabel(f'Time for test case')
    plt.title(f'{testcases[idx]['name']}')

    # Save the chart to a file (e.g., as a PNG image)
    plt.savefig(f'{testcases[idx]['name']}_{pictureName}.png')

    # avoid overlap
    plt.close()

def Rechunk(chunkSchemes) -> None:
  print("Attempting to re-chunk dataset")

SyntaxError: f-string: unmatched '[' (3089549306.py, line 10)

In [15]:
def Rechunk(chunkSchemes) -> None:
  print("Attempting to re-chunk dataset")

  rrt = None
  for scheme in chunkSchemes:
    if not os.path.exists(f"/anvil/scratch/{USERNAME}/{scheme['name']}.zarr"):

      if rrt is None:
        print("Missing scheme found! Loading entire dataset...")
        ds = xr.open_mfdataset("/anvil/projects/x-cis220065/x-cybergis/compute/AORC_Forcing/2016/*.LDASIN_DOMAIN1", engine = "netcdf4", combine= "nested", concat_dim="Time", parallel = "True")
        rrt = ds["RAINRATE"]

      print(f"Re-chunking using scheme {scheme['name']}")

      rrt = rrt.chunk(scheme['scheme'])
      rrt.to_zarr(f"/anvil/scratch/{USERNAME}/{scheme['name']}.zarr", mode="w")

In [16]:
if __name__ == "__main__" :
  #Start the client
  initialize()

  # Test chunk schemes across time
  chunkSchemes = [
    {'name': "10x100x100", 'scheme': {"Time": 10, "south_north": 100, "west_east": 100}},
    {'name': "50x300x300", 'scheme': {"Time": 50, "south_north": 300, "west_east": 300}},
    {'name': "100x600x600", 'scheme': {"Time": 100, "south_north": 600, "west_east": 600}},
    {'name': "150x900x900", 'scheme': {"Time": 150, "south_north": 900, "west_east": 900}},
    {'name': "200x1200x1200", 'scheme': {"Time": 200, "south_north": 1200, "west_east": 1200}},
    ]

  Rechunk(chunkSchemes)
  RunTestCases("chunk_test", testcases, chunkSchemes)

  shutdown()

Attempting to re-chunk dataset
Running benchmark test cases
	Testing scheme: 10x100x100
		Test case: Max_Rain_Test


/home/x-twhite77/.conda/envs/rocky8.7/conda-4.12.0/labpacks/lib/python3.9/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 12.86 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/x-twhite77/.conda/envs/rocky8.7/conda-4.12.0/labpacks/lib/python3.9/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 12.86 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/x-twhite77/.conda/envs/rocky8.7/conda-4.12.0/labpacks/lib/python3.9/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 12.86 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/x-twhite77/.conda/envs/rocky8.7/conda-4.12.0/labpacks/lib/python3.9/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 12.86 MiB.
This may caus

		Test case: Average_200_sqr
		Test case: Average_400_sqr


2024-04-19 10:41:36,375 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)


		Test case: Average_800_sqr


...

2024-04-19 10:46:35,071 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)


	Testing scheme: 150x900x900
		Test case: Max_Rain_Test
		Test case: Average_200_sqr
		Test case: Average_400_sqr
		Test case: Average_800_sqr
		Test case: Average_1600_sqr
	Testing scheme: 200x1200x1200
		Test case: Max_Rain_Test


2024-04-19 10:46:48,381 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.59 GiB -- Worker memory limit: 3.69 GiB
2024-04-19 10:46:48,391 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.73 GiB -- Worker memory limit: 3.69 GiB
2024-04-19 10:46:48,403 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

KilledWorker: Attempted to run task ('nanmax_skip-b7144da7a57c136e5833ac58895447f2', 0, 2, 2) on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:36269. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

2024-04-19 10:47:05,617 - distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)


In [22]:
plt.plot(.groupby(df['date'].dt.year)['stars'].mean())

NameError: name 'dataset' is not defined